Period based on mass of central object:
$$T = 2\pi\sqrt{\frac{r^{3}}{GM}}$$
Lorentz factor for velocity:
$$\gamma_{velocity} = \frac{1}{\sqrt{1-(\frac{v}{c})^{2}}}$$ 

$\Delta t' = \Delta t\gamma_{v}$, Where **t'** is the elapsed time for an observer moving at speed **v**, and **t** is the elapsed time for a clock at rest.

Gravitational Time Dilation Formula
$$t_{0} = t_{f}\sqrt{1-\frac{2GM}{rc^{2}}} = t_{f}\sqrt{1-\frac{r_{s}}{c^{2}}}$$

Where $t_{0}$ is the elapsed time for an observer close to a gravitational object $t_{f}$ is the elapsed time for an observer at a large distance.

Current problems in the code (first time using pygame):
1. Pause button isn't displaying
2. Texts describing the values of distance, relative and inertial time, and the gamma factors for each planet currently print on top of each other when more than one planet is added to the list of planets.
3. There is no functionality for light - light acts as an object which can be slowed down by gravity instead of having a constant speed_magnitude of c
4. There is no functionality to measure length contraction between objects
5. There is no functionality to choose a frame of reference
6. There is no functionality allowing the user to set scale, timesteps, choose planets, change bh mass, set original velocities, etc outside of manipulating direct code
7. Objects inside the photon sphere can still have stable orbits

In [2]:
# Base units are in kg, m, and s
import numpy as np
import math

# Creating a function to calculate escape velocity
def calc_escape_velocity(mass, distance):
    return np.sqrt((2 * G * mass)/distance)

# Creating a function to find the distances of chosen escape velocities
def calc_esc_radius(mass, V_esc):
    # Should give swarzchild radius when V_esc = c
    return (2 * G * mass)/V_esc **2

def calc_radial_velocity(radius, period):
    return (2 * np.pi * radius)/period

def calc_radius_from_vel(rad_vel, period):
    return (rad_vel * period)/ (2 * np.pi)
   

def calc_period(distance, mass):
    return 2 * np.pi *(math.sqrt(distance**3/(G * mass)))
   

def calc_grav_time_dilation(s_radius, distance):
    return (1 - (s_radius / distance))**0.5
    

def calc_vel_time_dilation(velocity):
    return 1/(math.sqrt(1-(velocity/c)**2))
        
    
def rel_velocity_addition(v1, v2):
    return (v1 + v2 ) / (1 + (v1 * v2)/(c**2))

# Setting Constants
G          = 6.674e-11      # N * m^2 * kg^-2
c          = 2.99e+8        # m * s^-1
sol_mass   = 1.99e+30       # kg
earth_mass = 5.97e+24       # kg
moon_mass  = 7.348e+22      # kg
moon_dist  = 3.84e+8        # m
jup_mass   = 1.89e+27       # kg
au         = 1.49e+11       # m
year       = 3600*24*365.25 # s
second     = 1
lightyear  = c * year       # m
moon_speed = calc_radial_velocity(moon_dist, 3600*24*27.32)

bh_mass    = 100640 * sol_mass # Intersteller gargantua = 1e8, TON618 = 6.6e10, 1.012e5 = s_radius is 1 light second
s_radius   = calc_esc_radius(bh_mass, c)

print(f"{calc_esc_radius(bh_mass, c)}m which is {calc_esc_radius(bh_mass, c)/c}light seconds across")
print(f"G * M = {G * bh_mass}")

299018133.2199863m which is 1.00006064622069light seconds across
G * M = 1.3366260064e+25


In [ ]:
import pygame
import sys
import math
pygame.init()

time_scales = [
    (1e9 * year, "billion year"),
    (1e6 * year, "million year"),
    (1e3 * year, "thousand year"),
    (year, "year"),
    (1, "second"),
    (1e-6, "microsecond"),
    (1e-9, "nanosecond"),
    (1e-12, "picosecond"),
    (5.39e-44, "Planck second")  
]

# Creating a figure and axis
width, height = 1280, 700
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orbit Simulation")

# Color library
white        = (255,255,255)
black        = (0, 0, 0)
dark_gray    = (89, 89, 89)
yellow       = (255, 255, 0)
blue         = (0, 0, 255)
red          = (255, 0, 0)
cosmic_latte = (255, 248, 231)
gray         = (222, 222, 222)
light_blue   = (204,212,255)
brown        = (77,38,0)

# Font
font         = pygame.font.SysFont("ariel", 16)
pause_bttn   = pygame.Rect(20, 20, 100, 50)

# Time and space scaling
# Still needs to handle whether or not a black hole is the central object
# So that this can also be set for the solar system, just for fun
# For now, comment out the one not being used

n           = 1.001 # changing this value might make some of the tests fail or not work as intended
orbit_n     =  n * calc_esc_radius(bh_mass, c)
bh_period   = calc_period(orbit_n, bh_mass)
bh_orbit_speed  = calc_radial_velocity(orbit_n, bh_period)
escape_v_n      = calc_escape_velocity(bh_mass, orbit_n)
print(f"s_radius = {s_radius/c:.5f}light seconds or {s_radius:.5f}m")
print(f"8 * orbit_n = {8 * orbit_n:.5e}m")

dt            = bh_period/36
scale         = 75 / orbit_n

#dt            = year/360
#scale            = 800 / au

object_texts = {}

class Planet: 
    def __init__(self, x, y, radius, color, mass):
        self.x               = x
        self.y               = y
        self.radius          = radius # in pixels
        self.color           = color
        self.mass            = mass
        self.rel_clock       = 0
        self.intl_clock      = 0
        
        # Should also make light object functionality
        self.sun             = False
        self.distance_to_sun = 1 # Avoiding division by 0
        
        # To draw event horizons and handle collisions
        self.bh              = False
        self.absorbed        = False
        
        self.orbit           = []
        
        self.x_vel           = 0 # m/s
        self.y_vel           = 0 # m/s
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        self.light_clock_circles = []
                
            
    def draw_event_horizon(self, planets):
        self.event_horizon = calc_esc_radius(self.mass, c)
        center             = (scale * self.x + (width/2), scale * self.y + (height/2))
        for planet in planets:
            if self.bh:  
                # Draw event horizon
                pygame.draw.circle(win, black, center, int(scale*self.event_horizon), 2)
                
                # Draw photon sphere
                pygame.draw.circle(win, yellow, center, int(scale * 1.5 * self.event_horizon),2)
    
    # This isn't working, I have no idea why not.    
    def relative_velocities(self, planets):
        for other in planets:
            if self != other:

                rel_vx             = rel_velocity_addition(self.x_vel, other.x_vel)
                rel_vy             = rel_velocity_addition(self.y_vel, other.y_vel)
                rel_speed          = math.sqrt(rel_vx**2 + rel_vy**2)
                
                classic_speed      = math.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2)/ dt
                
                # Only show info for first planet
                if self == planets[0]:
                    # Change depending on readability
                    if rel_speed <= c / 1000: 
                        rel_speed_text = font.render(f"Relative speed: {rel_speed:.3e}m/s", 1, self.color)
                    else:
                        rel_speed_text = font.render(f"Relative speed: {rel_speed / c:.5f}c", 1, self.color)
                    
                    classic_speed_text = font.render(f"Classic speed: {classic_speed:.3e}m/s", 1, self.color)
                    
                    
                    rel_text_rec = rel_speed_text.get_rect()
                    classic_rec  = classic_speed_text.get_rect()

                    win.blit(rel_speed_text, ((width - rel_text_rec.width) / 2, 10))   
                    win.blit(classic_speed_text, ((width - classic_rec.width) / 2, 30))
                    
    def light_clock(self, planets, is_1d = False):
        if not self.bh and not self.sun:
            if is_1d:
                center = scale * self.x + width/2, height/2
            
            else:
                center = (scale * self.x + (width/2), scale * self.y + (height/2))
            
            # Find gamma factors for gravity and velocity
            self.grav_gamma     = calc_grav_time_dilation(s_radius, self.distance_to_sun)
            self.vel_gamma      = calc_vel_time_dilation(self.speed_magnitude)
            
            # Set inertial and relative clocks
            self.rel_clock     += self.grav_gamma * self.vel_gamma * dt# / (self.vel_gamma)
            self.intl_clock    += dt
            
            
            if int(self.rel_clock % 1) == 0:
                # Draw a circle with radius 0
                pygame.draw.circle(win, white, center, 0, 1)  
                # Store the center and the current time when the circle was drawn
                self.light_clock_circles.append((center, self.rel_clock))
                
            for circle_data in self.light_clock_circles[:]:  # Iterate over a copy of the list to allow removal
                circle_center, draw_time = circle_data
                # Calculate the elapsed time since the circle was drawn
                #elapsed_time = self.rel_clock - draw_time
                # Calculate the radius based on the growth rate
                radius = scale * c * self.intl_clock
                # Draw the updated circle
                pygame.draw.circle(win, white, circle_center, max(1, int(radius)), 1)
                # Remove the circle from the list if it's too large
                if radius >= width/3:
                    self.light_clock_circles.remove(circle_data)
                 

    
    def attraction(self, other):
        other_x, other_y = other.x, other.y
        distance_x       = other_x - self.x
        distance_y       = other_y - self.y
        distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
        
        if not self.sun and not self.bh:
            print(f"x_distance from bh = {distance_x}, y_distance from bh = {distance_y}")
        
        if other.sun:
            self.distance_to_sun = distance
        elif other.bh:
            self.distance_to_sun = distance
        
        mu      = self.mass * other.mass
        force   = G * mu / distance ** 2
        theta   = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y
    
    
    def update_position(self, planets):
        # Initial forces at 0
        total_fx = total_fy = 0
        
        for planet in planets:
            # Avoiding division by 0
            if self == planet:
                continue
            fx, fy    = self.attraction(planet)
            total_fx += fx
            total_fy += fy
            
        # Calculate acceleration
        accel_x = total_fx / self.mass
        accel_y = total_fy / self.mass
    
        # Update velocity components
        self.x_vel += accel_x * dt
        self.y_vel += accel_y * dt
        
        self.x += self.x_vel * dt
        self.y += self.y_vel * dt
        
        
        self.orbit.append((self.x, self.y))
        
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        if self.speed_magnitude >= c:
            # Cap velocity
            self.x_vel = self.x_vel / self.speed_magnitude * (c * 0.9999)
            self.y_vel = self.y_vel / self.speed_magnitude * (c * 0.9999)
    
    # Have things fall into the black hole
    def collide(self, other, planets):
        other.event_horizon    = calc_esc_radius(other.mass, c)
        distance_x, distance_y = self.x - other.x, self.y - other.y
        distance               = math.sqrt(distance_x**2 + distance_y**2)
        
        if distance <= other.event_horizon:
            other.mass += self.mass
            planets.remove(self)
                        
                
    def draw(self, planets, win, is_1d=False):
        if not self.sun:
            if not self.bh:
                # Find gravitational dilation factor
                self.grav_gamma     = calc_grav_time_dilation(s_radius, self.distance_to_sun)
                grav_gamma_text     = font.render(f"Gγ: {self.grav_gamma:.3f}", 1, self.color)

                # Find lorentz factor
                self.vel_gamma      = calc_vel_time_dilation(self.speed_magnitude)
                vel_gamma_text      = font.render(f"Vγ: {self.vel_gamma:.3f}", 1, self.color)

                # Update inertial and relative time counters
                self.rel_clock     += self.grav_gamma * self.vel_gamma * dt# / (self.vel_gamma)
                self.intl_clock    += dt

                time_diff           = self.intl_clock - self.rel_clock


                if self.speed_magnitude <= c/1000:
                    vel_text        = font.render(f"Velocity of planet: {self.speed_magnitude:.3f}m/s",1, self.color)
                else:
                    vel_text        = font.render(f"Velocity of planet: {self.speed_magnitude/c:.3f}c",1, self.color)

                # Change the time scale as the counters gets higher
                for exponent, unit in time_scales:
                    # Check if time_counter or relative_time is greater than the current scale
                    if self.rel_clock >= exponent:
                        # Use the current scale for rendering
                        if unit == "Planck second":
                            rel_time_text   = font.render(f"Time in {unit}s: {self.rel_clock/exponent:.3e}", 1, self.color)
                            time_diff_text  = font.render(f"Difference in times: {time_diff/exponent:.3e} {unit}s", 1, self.color)
                            intl_clock_text = font.render(f"Distant observer clock in {unit}s: {self.intl_clock/exponent:.3e} {unit}s", 1, self.color)
                        else:
                            rel_time_text   = font.render(f"Clock on planet in {unit}s: {self.rel_clock/exponent:.3f}", 1, self.color)
                            time_diff_text  = font.render(f"Difference in times: {time_diff/exponent:.3f} {unit}s", 1, self.color)
                            intl_clock_text = font.render(f"Distant observer clock in {unit}s: {self.intl_clock/exponent:.3f} {unit}s", 1, self.color)
                        break

                #print(f"Debugging: dt: {dt:.3f} Vγ: {self.vel_gamma:.3f}, Gγ: {grav_gamma:.3f}, Rel_clock: {self.rel_clock:.5f})")#, Inertial clock: {self.intl_clock:.5f}")

                if self.distance_to_sun >= lightyear:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius)/lightyear:.2f}lyrs from EH", 1, self.color)
                elif self.distance_to_sun >= au/1000:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius)/au:.3e}au from EH", 1, self.color)
                else:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius):.3e}m from EH", 1, self.color)

                object_texts[self] = {
                    "intl_clock": intl_clock_text,
                    "rel_time": rel_time_text,
                    "time_diff": time_diff_text,
                    "distance": distance_text,
                    "vel": vel_text,
                    "grav_gamma": grav_gamma_text,
                    "vel_gamma": vel_gamma_text
                }

            for obj, texts in object_texts.items():
                y_offset = 10
                for text_name, text_surface in texts.items():
                    win.blit(text_surface, (10, y_offset))
                    y_offset += 20
                    
            #print(f"x_vel = {self.x_vel/c:.5f}c, y_vel = {self.y_vel/c:.5f}c, speed_magnitude = {self.speed_magnitude/c:.5f}c")
            #print(f"x_pos = {self.x:.5f}, y_pos = {self.y:.5f}")
            #print(f"intl_clock = {self.intl_clock:5f}, rel_clock = {self.rel_clock:5f}")
            #print(f"vel_gamma = {self.vel_gamma:.5f}. grav_gamma = {self.grav_gamma:.5f}")
            #print()
            # The values are printed to allow for my paper to be written easier.
            # In the future, it would make more sense to populate a list with their values
            # And have a function which grabs specific mins, maxes, averages, etc depending on need
        if is_1d:
            center_x = width/2
            
            for planet in planets:
                x = center_x + planet.x * scale
                pygame.draw.circle(win, planet.color, (int(x), height // 2), planet.radius)
            
        else:
            x = self.x * scale + width/2
            y = self.y * scale + height/2


            if len(self.orbit) > 2:
                # Keep only the most recent points
                if len(self.orbit) > 30000:
                    self.orbit = self.orbit[-30000:]

                updated_points = []

                for point in self.orbit:
                    x, y = point
                    x = x * scale + width/2
                    y = y * scale + height/2
                    updated_points.append((x,y))

                pygame.draw.lines(win, self.color, False, updated_points, 1)

            pygame.draw.circle(win, self.color, (x,y), self.radius)
            
             
def main():
    run            = True
    paused         = False
    clock          = pygame.time.Clock()
    time_counter   = 0
    relative_time  = 0
    time_diff      = 0
    light_clock    = 0
    
    
    sun            = Planet(0,0, int(scale * 6.96e8), yellow, sol_mass)
    sun.sun        = True
    
    mercury        = Planet(0.39 * au, 0, int(scale * 2.4e6), white, 0.33e+24)
    mercury.y_vel  = -calc_radial_velocity(0.39*au, 0.24*year)
    
    venus          = Planet(0.72 * au, 0, int(scale * 6.1e6), brown, 4.87e+24)
    venus.y_vel    = -calc_radial_velocity(0.72*au, 0.62*year)
    
    earth          = Planet(au, 0, int(scale * 6.34e6), blue, earth_mass)
    earth.y_vel    = -calc_radial_velocity(au, year)
    
    moon           = Planet(au + moon_dist, 0, 1, gray, moon_mass)
    moon.y_vel     = -(moon_speed-earth.y_vel)
    
    mars           = Planet(1.524 * au, 0, int(scale * 3.4e6), red, 0.11 * earth_mass)
    mars.y_vel     = -2.41e4
    
    jupiter        = Planet(-5.2 * au, 0, int(scale * 6.9e7), red, jup_mass)
    jupiter.y_vel  = calc_radial_velocity(5.2 * au, 12* year)
    
    saturn         = Planet(-9.54 * au,0, int(scale * 5.8e7), yellow, 568e+24)
    saturn.y_vel   = calc_radial_velocity(9.54 * au, 28.7*year)
    
    uranus         = Planet(-19.2*au, 0, int(scale * 2.5e7), blue,8.68e25)
    uranus.y_vel   = calc_radial_velocity(19.2*au, 2.65e9) # had the value in seconds
    
    neptune        = Planet(-30.1*au, 0, int(scale * 2.5e7), blue, 1.02e26)
    neptune.y_vel  = calc_radial_velocity(30.1*au, 5.2e9)
    
   
    
    # calc_radial_velocity(distance, period), calc_period(distance, mass)
    
    black_hole     = Planet(0, 0, 3, black, bh_mass)
    black_hole.sun = True
    black_hole.bh  = True
    
    t_bh           = Planet(0,0,3, black, bh_mass/10)
    t_bh.sun       = True
    t_bh.bh        = True
    
    t1             = 1.004316016 # Optimized to stay as close to BH as possible
    t_plnt         = Planet(-t1*orbit_n, 0, 2, white, 2)
    t_plnt.y_vel   = -bh_orbit_speed
    
    t2 = 8
    t2_plnt        = Planet(t2 * orbit_n, 0, 2, white, 1)
    t2_plnt.y_vel  = -0.475*calc_radial_velocity(t2 * orbit_n, calc_period(t2 * orbit_n, bh_mass))
    
    t3 = 7
    t3_plnt        = Planet(-t3 * orbit_n, 0, 2, white, 2)
    t3_plnt.y_vel  = -calc_radial_velocity(t3 * orbit_n, calc_period(t3*orbit_n, t_bh.mass))
    
    t4 = 6
    t4_plnt        = Planet(t4 * orbit_n, 0, 2, white, 2)
    t4_plnt.y_vel  = 0.55*calc_radial_velocity(t4*orbit_n, calc_period(t4 * orbit_n, bh_mass))
    
    t5_plnt        = Planet(6 * orbit_n, 0, 2, white, 2)
    t5_plnt.x_vel  = -0.9999 * c
    
    t6_plnt        = Planet(-6 * orbit_n, 0, 2, white, 2)
    t6_plnt.x_vel  = 0.9999 * c
    
    t7 = 7
    t7_plnt        = Planet(-c, 0, 2, white, 2)
    t3_plnt.y_vel  = -calc_radial_velocity(t7 * orbit_n, calc_period(t7*orbit_n, t_bh.mass))
    
    
    rel_speed_test        = [t5_plnt, t6_plnt] # Currently not working
    close_orbit_test      = [black_hole, t_plnt] # fails if dt > orbit_n / 360 - planet falls in with bigger timesteps
    elliptical_orbit_test = [black_hole, t2_plnt]
    wide_orbit_test       = [t_bh, t3_plnt]
    
    planets = elliptical_orbit_test
    
    while run:
        clock.tick(30)
        win.fill(dark_gray)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if pause_bttn.collidepoint(event.pos):
                        paused = not paused
        if not paused:
            # Change the time scale as the counters gets higher
            for exponent, unit in time_scales:
                # Check if time_counter or relative_time is greater than the current scale
                if time_counter >= exponent:
                    # Use the current scale for rendering
                    if unit == "Planck second":
                        time_text     = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3e}", 1, white)
                    else:
                        time_text = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3f}", 1, white)
                    break

            for planet in planets:
                planet.update_position(planets)
                planet.draw(planets, win, is_1d=False)
                planet.draw_event_horizon(planets)
                #planet.light_clock(planets, is_1d=True)
                for other in planets:
                    if planet != other:
                        planet.collide(other, planets)
                        

    

            
        pygame.display.update()
                
    pygame.quit
    
main()

s_radius = 1.00006light seconds or 299018133.21999m
8 * orbit_n = 2.39454e+09m
x_distance from bh = -2394537210.82565, y_distance from bh = 0.0
x_distance from bh = -2394394746.421989, y_distance from bh = 8773191.052799022
x_distance from bh = -2394109803.5303783, y_distance from bh = 17545860.05675979
x_distance from bh = -2393682356.8449206, y_distance from bh = 26317484.684384637
x_distance from bh = -2393112369.8274903, y_distance from bh = 35087542.1437539
x_distance from bh = -2392399794.6923594, y_distance from bh = 43855508.992313534
x_distance from bh = -2391544572.386047, y_distance from bh = 52620860.950037464
x_distance from bh = -2390546632.562378, y_distance from bh = 61383072.71178819
x_distance from bh = -2389405893.5527234, y_distance from bh = 70141617.75869758
x_distance from bh = -2388122262.331392, y_distance from bh = 78895968.16838807
x_distance from bh = -2386695634.4761386, y_distance from bh = 87645594.42385243
x_distance from bh = -2385125894.123753, y_dista

x_distance from bh = -1549586977.6004453, y_distance from bh = 788781319.7365249
x_distance from bh = -1531756140.8119464, y_distance from bh = 793261933.9646612
x_distance from bh = -1513681528.4504719, y_distance from bh = 797616302.3255086
x_distance from bh = -1495360053.251652, y_distance from bh = 801840589.2790029
x_distance from bh = -1476788531.1355696, y_distance from bh = 805930796.3050249
x_distance from bh = -1457963677.3408113, y_distance from bh = 809882752.1209242
x_distance from bh = -1438882102.3717158, y_distance from bh = 813692102.1542302
x_distance from bh = -1419540307.7501862, y_distance from bh = 817354297.2012715
x_distance from bh = -1399934681.5633495, y_distance from bh = 820864581.1948092
x_distance from bh = -1380061493.7983723, y_distance from bh = 824217977.9951968
x_distance from bh = -1359916891.455887, y_distance from bh = 827409277.1098818
x_distance from bh = -1339496893.4337897, y_distance from bh = 830433018.2350905
x_distance from bh = -13187973

x_distance from bh = -898501008.135424, y_distance from bh = -860051582.3932511
x_distance from bh = -924847010.5527213, y_distance from bh = -861889297.2318238
x_distance from bh = -950819096.3853954, y_distance from bh = -863378549.3531395
x_distance from bh = -976424553.0042567, y_distance from bh = -864534888.7258835
x_distance from bh = -1001670427.0746385, y_distance from bh = -865372850.5469968
x_distance from bh = -1026563529.4376577, y_distance from bh = -865906042.4061136
x_distance from bh = -1051110440.8893913, y_distance from bh = -866147222.3309349
x_distance from bh = -1075317518.5872173, y_distance from bh = -866108368.8258224
x_distance from bh = -1099190902.872119, y_distance from bh = -865800743.8624568
x_distance from bh = -1122736524.3426561, y_distance from bh = -865234949.6513785
x_distance from bh = -1145960111.053312, y_distance from bh = -864420979.9127293
x_distance from bh = -1168867195.7391608, y_distance from bh = -863368266.2703532
x_distance from bh = -1

x_distance from bh = -2339555400.6357155, y_distance from bh = -262261387.067063
x_distance from bh = -2343463073.8997297, y_distance from bh = -253720063.09680998
x_distance from bh = -2347224582.7430563, y_distance from bh = -245162914.3216124
x_distance from bh = -2350840219.594678, y_distance from bh = -236590529.5085566
x_distance from bh = -2354310264.170971, y_distance from bh = -228003492.16691753
x_distance from bh = -2357634983.6010847, y_distance from bh = -219402380.79207528
x_distance from bh = -2360814632.546163, y_distance from bh = -210787769.1044107
x_distance from bh = -2363849453.312531, y_distance from bh = -202160226.28345302
x_distance from bh = -2366739675.9589705, y_distance from bh = -193520317.19754395
x_distance from bh = -2369485518.398198, y_distance from bh = -184868602.62927425
x_distance from bh = -2372087186.492646, y_distance from bh = -176205639.49693972
x_distance from bh = -2374544874.144654, y_distance from bh = -167531981.07225686
x_distance from 

x_distance from bh = -1984651313.9625957, y_distance from bh = 594134229.7117829
x_distance from bh = -1973057878.0581803, y_distance from bh = 601248665.9976388
x_distance from bh = -1961280778.5468547, y_distance from bh = 608307134.5899593
x_distance from bh = -1949318651.45826, y_distance from bh = 615308215.3790748
x_distance from bh = -1937170096.8706424, y_distance from bh = 622250449.7750491
x_distance from bh = -1924833677.858466, y_distance from bh = 629132339.0712991
x_distance from bh = -1912307919.3960986, y_distance from bh = 635952342.7238452
x_distance from bh = -1899591307.2154474, y_distance from bh = 642708876.5407916
x_distance from bh = -1886682286.6152995, y_distance from bh = 649400310.7762324
x_distance from bh = -1873579261.220003, y_distance from bh = 656024968.1223391
x_distance from bh = -1860280591.6849835, y_distance from bh = 662581121.5929058
x_distance from bh = -1846784594.3464546, y_distance from bh = 669066992.2911018
x_distance from bh = -1833089539